# Part 1: Anna

In [59]:
import sympy as sym
from sympy.solvers.solveset import nonlinsolve
import numpy as np
from scipy.integrate import ode
import matplotlib.pyplot as plt

## Introduction and Background
This project focuses on using epidemic models to explore various aspects of python. Even though Alex and I worked together on this project, we focused on different implementations and features of python. The primary focus of my portion of the project was to explore the python library SymPy through the lens of stability and equilibrium analysis of epidemic models. The second part of the project I worked on looked at modifying the SIR.py code given in class to accommodate simulations where there is a parameter change during a run. Since this project relies on epidemic models and modeling we will start with a brief overview of mathematical epidemic modeling and the specific models used in this project.

There are many different variants of mathematical epidemic models which all serve the same purpose, to track and predict disease spread in a community. Some of the most common models are the susceptible-infected-recovered (SIR), susceptible-infected-susceptible (SIS), susceptible-infected (SI), or susceptible-exposed-infected (SEI). These models work by dividing a population into classes, where the specific classes depend on the disease you are studying and which model is used. For example, a SIS model divides a population into a susceptible class and an infected class, while a SEI model divides the population into a susceptible class, exposed class, and infected class. Even though the exact classes vary between models, all mathematical epidemic models have a susceptible class and an infected class. The susceptible class, denoted by $S$, is defined to be the group of individuals who are not infected with the disease, but are susceptible to infection and the infected class, denoted by $I$, are the individuals who are not only infected with the disease, but are capable of transmitting it to others in the population. Another important aspect to these models is movement. The most basic way for movement between the classes to happen in these models are from interactions between the classes or interaction with the disease. Thus, when the infected and susceptible classes come into contact there is the possibility of transmission, so those who contract the disease would moved from the susceptible class to the infected class.

Mathematically, epidemic models are described as a dynamical system, often built by ordinary differential equations (ODEs); although some more complicated models can include partial differential equations (PDEs). For this project we are only using are ODE models. Once these models are described mathematically many kinds of analysis that can be preformed. In the  681Project\_SymPy\_Sisk.py code I use python to explore equilibrium and stability analysis, along with calculating the basic reproductive number, $\mathscr{R}_0$. Briefly (and simplistically), equilibrium and stability analysis looks at the long term behavior of the system and the basic reproductive number is the number of secondary infections caused by one infectious individual. Mathematically, $\mathscr{R}_0$ tells us if the differential equation describing the $I$ class will be positive or negative. In other words, if the population of the infected class is increasing or decreasing. It also has the condition, if $\mathscr{R}_0>1$ then the disease will persist in the population and if $\mathscr{R}_0<1$, the disease will eventually be eliminated. 

In my portion of the project I look at three different epidemic models. Model 1 in the 681Project\_ SymPy\_Sisk.py code comes from chapter 6, page 273 of Linda Allen's book, *An Introduction to Mathematical Biology*. It is a simple SIS model that does not allow for births and deaths, where $N$ is the total population, $\gamma$ is the recovery rate, and $\beta$ is the contact rate. While model 1, seen below, is a general model, Allen notes that models of this form are often used to study the spread of sexually transmitted diseases since no immunity is gained after recovery.

\begin{equation}
\begin{split}
    \frac{dS}{dt}&=-\frac{\beta}{N}SI+\gamma I\\
    \frac{dI}{dt}&=\frac{\beta}{N}SI-\gamma I
\end{split}
\end{equation}

Model 2 in the 681Project\_SymPy\_Sisk.py code, seen below, is from chapter 2, page 35 of Fred Brauer and Carlos Castillo-Chavez's book, *Mathematical Models for Communicable diseases*.
\begin{equation}
\begin{split}
    S'&=\lambda-\beta SI-\mu S\\
    I'&=\beta SI-\mu I-\alpha I\\
    N'&=\lambda-\mu N
\end{split}
\end{equation}
This is a general SI model, where $N$ is the total population, $\alpha$ is the rate of departure from the infective class through recovery, $\beta$ is the per-capita contact, $\mu$ is the proportional natural death rate, and $\lambda$ is the density-dependent birth rate. Unlike model 1, this model allows for births and death, but no deaths caused by the disease. It also assumes everyone is born susceptible and there is full immunity after recovery. Further, the authors then note that since the model is asymptotically autonomous we can consider $N$ to be constant and perform analysis on the follow system instead,
\begin{equation}\label{BCModelRed}
\begin{split}
    S'&=\lambda-\beta SI-\mu S\\
    I'&=\beta SI-\mu I-\alpha I.\\
\end{split}
\end{equation}
This reduced system is used in the 681Project\_SymPy\_Sisk.py.

The last model I looked at is an SEIR model from Hem Joshi, Suzanne Lenhart, Micheal Li, and Liancheng Wang's paper *Optimal Control Methods Applies to Disease Models*. I used this model in the 681Project\_NumSim\_Sisk.py code as a part of the modification the SIR.py code. The model,
\begin{equation}
\begin{split}
    S'&=bN-dS-\frac{\beta IS}{N}-vS\\
    E'&=\frac{\beta IS}{N}-(\epsilon+d)E\\
    I'&=\epsilon E-(\gamma+\alpha+d)I\\
    R'&=\gamma I-dR+vS\\
    N'&=(b-d)N-\alpha I
\end{split}
\end{equation}
is designed to study a micro-parasitic infectious disease, where $\alpha$ is the death rate caused by the disease, $b$ is the natural birth rate, $\beta$ is the adequate contact rate between susceptible and infectious individuals, $d$ is the natural death rate, $\epsilon$ is the rate exposed individuals become infectious, $\gamma$ is the recovery rate, and $v$ is the constant per capita vaccination rate. This model is more complicated then models 1 and 2 since they disease requires an incubation period in the host before they become infectious (able to transmit the disease to other). This is seen by the inclusion of the exposed class. For my part of the project I am assuming a constant vaccination rate in this model even though the main focus of the paper is to find the optimal vaccination rate using optimal control techniques. Alex will go into the detail of the optimal control problem and implementation in python in her section of the project.

Now that we have some background on mathematical epidemic modeling and the particular models we are working with we will go through a brief introduction into the SymPy library. SymPy is python's symbolic library. According to the SymPy website the aim is "to become a full-featured computer algebra system (CAS) while keeping the code as simple as possible in order to be comprehensible and easily extensible". Since SymPy is is written entirely in python, once a user has gained familiarity python's syntax SymPy is relatively easy to learn how to use and a great alternative to other symbolic mathematical programs like Maple and Mathematica, where the user would need to learn a new syntax. SymPy has a wide range of features and modules ranging from basic algebraic manipulation and simplification of symbolic expressions up to more complicated tasks like solving ODEs/PDEs and computations on symbolic matrices. To get famariliar with the basic of SymPy, let's look at some examples. 

We will start with some simple expression manipulation. Note, unlike in other CAS programs (like Maple) the user must first initialize the symbols that will be used.

In [54]:
x = sym.Symbol('x')
y = sym.Symbol('y')

Now that we have initalized $x$ and $y$ symbols we can use them in a symbolic expression. The command pprint, short for pretty print, formats symbolic expressions (which is unnecessary in a Jupyter Notebook enviroment) and the command expand foils out the expression, as expected.

In [55]:
sym.pprint((x+y)**2)
sym.expand((x+y)**2)

       2
(x + y) 


x**2 + 2*x*y + y**2

Going one step further we can use solve command to find solutions to sybolic expressions. Note, the solve command is used to find roots and thus only requires the non-zero side of the equation.

In [56]:
sym.solve((x+y)**2,x)

[-y]

Now that we have seen some of the basic functions of SymPy, let's look at look at a more sophisticated feature-solving an ODE. While SymPy can handle ODE and PDE systems, in this example we will look at the differential equation for exponential growth. First, we define the symbolic function, $P(x)$, similar to how we defined other symbols, and then define $r$, the intrinsic growth, as before. Like the the solve command, dsolve requires the differential equation to be set equal to zero. Hence, we rearrange the differential equation as follows,
\begin{equation}
\begin{split}
    P'&=rP\\
    \implies P'-rP&=0
\end{split}
\end{equation}
and pass dsolve the left hand side.

In [57]:
P = sym.symbols('P', cls=sym.Function)
r = sym.Symbol('r')
sym.dsolve(P(x).diff(x)-r*P(x))

Eq(P(x), C1*exp(r*x))

While SymPy is able to solve various different types of ODEs and PDEs, there is a limit to the complexity it can handle. In addition to differential equations, SymPy also has modules dedicated to differential geometry, Lie algebra, category theory, logical and boolean expressions, and tensor operations to name a few. A full list of the modules and features of the SymPy library can be found on it's documentation page. For the 681Project\_SymPy\_Sisk.py code I explore the utility of the symbolic solvers, print formatting, and the ODE and linear algebra modules. Now that we have some background information we can eplore the 681Project\_ SymPy\_Sisk.py and 681Project\_NumSim\_Sisk.py codes.

## Methods and Results


## Discussion and Conclusion
What I learned/What I struggled with/What was interesting and helpful/Why this could be useful to other students

# Part 2: Alex